In [1]:
import pymysql
import pandas as pd

# ==================== 数据库配置 ====================
HOST = ""
PORT = 3306
USER = ""
PASSWORD = ""
DATABASE = "ionet"
CHARSET = "utf8mb4"

# ==================== 参数 ====================
POSITIVE_SAMPLES = 10000   # 正样本目标数量
NEGATIVE_SAMPLES = 10000   # 负样本目标数量
MIN_LENGTH = 40
MAX_LENGTH = 250
LENGTH_DIFF_MAX = 50       # 新增：一对序列长度差不超过 50 aa（推荐！）
OUTPUT_CSV = "protein_pair_20k_length_restricted_balanced.csv"

# ==================== 连接数据库 ====================
connection = pymysql.connect(
    host=HOST,
    port=PORT,
    user=USER,
    password=PASSWORD,
    database=DATABASE,
    charset=CHARSET
)

try:
    print(f"正在抽取正样本（label=1），目标 {POSITIVE_SAMPLES} 条，长度 40–250 aa...")
    pos_df = pd.read_sql("""
        SELECT sentence1, sentence2, label
        FROM protein_pair
        WHERE label = 1
          AND LENGTH(sentence1) BETWEEN %s AND %s
          AND LENGTH(sentence2) BETWEEN %s AND %s
          AND ABS(LENGTH(sentence1) - LENGTH(sentence2)) <= %s
        ORDER BY RAND()
        LIMIT %s
    """, connection, params=(MIN_LENGTH, MAX_LENGTH, MIN_LENGTH, MAX_LENGTH, LENGTH_DIFF_MAX, POSITIVE_SAMPLES))

    print(f"正在抽取负样本（label=0），目标 {NEGATIVE_SAMPLES} 条，长度 40–250 aa...")
    neg_df = pd.read_sql("""
        SELECT sentence1, sentence2, label
        FROM protein_pair
        WHERE label = 0
          AND LENGTH(sentence1) BETWEEN %s AND %s
          AND LENGTH(sentence2) BETWEEN %s AND %s
          AND ABS(LENGTH(sentence1) - LENGTH(sentence2)) <= %s
        ORDER BY RAND()
        LIMIT %s
    """, connection, params=(MIN_LENGTH, MAX_LENGTH, MIN_LENGTH, MAX_LENGTH, LENGTH_DIFF_MAX, NEGATIVE_SAMPLES))

    # 检查实际抽取数量
    actual_pos = len(pos_df)
    actual_neg = len(neg_df)
    print(f"\n实际抽取完成：")
    print(f"  正样本 (label=1): {actual_pos} 条")
    print(f"  负样本 (label=0): {actual_neg} 条")
    print(f"  总样本: {actual_pos + actual_neg} 条")

    if actual_pos < POSITIVE_SAMPLES:
        print(f"  警告：正样本不足，仅抽到 {actual_pos} 条（可能数据库中符合条件的样本较少）")
    if actual_neg < NEGATIVE_SAMPLES:
        print(f"  警告：负样本不足，仅抽到 {actual_neg} 条")

    # 合并并打乱
    sample_df = pd.concat([pos_df, neg_df], ignore_index=True)
    sample_df = sample_df.sample(frac=1, random_state=42).reset_index(drop=True)

    # 保存 CSV
    sample_df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")

    print(f"\n🎉 完成！共生成 {len(sample_df)} 条平衡样本（长度 40–250 aa，长度差 ≤ {LENGTH_DIFF_MAX} aa）")
    print(f"CSV 文件已保存至：{OUTPUT_CSV}")
    print("\n前5行预览：")
    print(sample_df.head())

except Exception as e:
    print(f"错误：{e}")
finally:
    connection.close()
    print("数据库连接已关闭。")

正在抽取正样本（label=1），目标 10000 条，长度 40–250 aa...


/tmp/ipykernel_9166/785285738.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pos_df = pd.read_sql("""


正在抽取负样本（label=0），目标 10000 条，长度 40–250 aa...


/tmp/ipykernel_9166/785285738.py:44: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  neg_df = pd.read_sql("""



实际抽取完成：
  正样本 (label=1): 10000 条
  负样本 (label=0): 10000 条
  总样本: 20000 条

🎉 完成！共生成 20000 条平衡样本（长度 40–250 aa，长度差 ≤ 50 aa）
CSV 文件已保存至：protein_pair_20k_length_restricted_balanced.csv

前5行预览：
                                           sentence1  \
0  MAKPAARPRKKVKKTVVDGIAHIHASFNNTIVTITDRQGNALSWAT...   
1  MKIGIIGAMEPEVAHLIAAMTNATSQTIAGIEFIAGTLAGKDVVVT...   
2  MSDSPRRFPTPLLLTGVMGVLILIALLTFTLMRSWDNAPDQSALLG...   
3  MKWSEVFHDITTRHDFQAMHDFLEKEYTTQIVYPDKKNIYQAFDLT...   
4  MSSVFEIVNQARRKNKLKRELLDNEKKVRDNRKRVDLLENLLDYIK...   

                                           sentence2  label  
0  MAALIVSRLARRGWLWKLPLATRREFWSRSRKEKEPVVAETVEEVK...      0  
1  MKIGIIGAMEPEVAHLIAAMTNATSQTIAGIEFIAGTLAGKDVVVT...      1  
2  MSDSPRRFPTPLLLTGVMGVLILIALLTFTLMRSWDNAPDQSALLG...      1  
3  MEWSQIFHDITTKHDFKAMHDFLEKEYSTAIVYPDRENIYQAFDLT...      1  
4  MSSGGLLLLLGLLTLWEVLTPVSSKDRPKFCELLPDTGSCEDFTGA...      0  
数据库连接已关闭。
